**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
import warnings
warnings.filterwarnings("ignore")
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-c76l95et
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-c76l95et
  Resolved https://github.com/Kaggle/learntools.git to commit 2bc4722ea2556ba6ed392c176ecd9157764adfd0
  Preparing metadata (setup.py) ... - done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268913 sha256=dd9202fb89e9b202815fe338fad5e3bbe4868523dc2310349afa3e55a5df1e66
  Stored in directory: /tmp/pip-ephem-wheel-cache-08irttb8/wheels/dd/d7/6b/0fc758f52767fd281d6dceded6757c6cb5bb90ccd2dbb1de9f
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,a0d581a6d12fe6a6399aedda09d80cc330472620,42560393a9c9b9ae28339f4b5aec77fd89bd49916ad541...,2017-06-08 23:30:00+00:00,2017-06-08 23:30:00+00:00,0,0.00,NaN,NaN,NaN,NaN,...,0.0,3.25,Cash,Patriot Taxi Dba Peace Taxi Associat,NaN,NaN,None,NaN,NaN,None
1,f0e3f6d8b0b8a7b21d298491d418329424835680,42560393a9c9b9ae28339f4b5aec77fd89bd49916ad541...,2017-06-08 23:30:00+00:00,2017-06-08 23:45:00+00:00,420,1.44,NaN,NaN,NaN,NaN,...,0.0,7.50,Cash,Patriot Taxi Dba Peace Taxi Associat,NaN,NaN,None,NaN,NaN,None
2,44664b713d15bc5f5e30946f007cd67127455091,42560393a9c9b9ae28339f4b5aec77fd89bd49916ad541...,2017-06-08 23:45:00+00:00,2017-06-09 00:15:00+00:00,1740,9.31,NaN,NaN,NaN,NaN,...,0.0,27.00,Cash,Patriot Taxi Dba Peace Taxi Associat,NaN,NaN,None,NaN,NaN,None
3,b2010387800513ecf68688d14f2c77cbffbdca57,e14b72f6ec0a3804a1a989e2e818a2b82e73ef8b3d462c...,2017-06-04 10:30:00+00:00,2017-06-04 10:45:00+00:00,720,2.60,NaN,NaN,NaN,NaN,...,0.0,13.25,Credit Card,Globe Taxi,NaN,NaN,None,NaN,NaN,None
4,adf6c8f64da007f48631ceca8400f8cf2f4bf8dc,e14b72f6ec0a3804a1a989e2e818a2b82e73ef8b3d462c...,2017-06-04 11:00:00+00:00,2017-06-04 11:15:00+00:00,1200,4.30,NaN,NaN,NaN,NaN,...,1.0,20.30,Credit Card,Globe Taxi,NaN,NaN,None,NaN,NaN,None


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-01-19,85156.571429
1,2016-01-23,83809.428571
2,2016-02-28,92816.428571
3,2016-03-29,75684.000000
4,2016-01-26,82071.285714


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
# q_1.hint()
# q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,67.0,2013-10-03 01:15:00+00:00,2013-10-03 01:30:00+00:00,1
1,67.0,2013-10-03 07:45:00+00:00,2013-10-03 07:45:00+00:00,2
2,67.0,2013-10-03 08:00:00+00:00,2013-10-03 08:00:00+00:00,3
3,67.0,2013-10-03 08:00:00+00:00,2013-10-03 08:00:00+00:00,3
4,67.0,2013-10-03 21:00:00+00:00,2013-10-03 21:00:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
# q_2.hint()
# q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://i.imgur.com/zzHgmlx.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [7]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,00752524a4118a162dce62ff49c0eb0f79530d9e0ecb5a...,2013-10-03 05:45:00+00:00,2013-10-03 06:00:00+00:00,315.0
1,00752524a4118a162dce62ff49c0eb0f79530d9e0ecb5a...,2013-10-03 19:30:00+00:00,2013-10-03 20:00:00+00:00,255.0
2,07780d6ed6024003f9fd2d0dae5b2af4e11e3303e73641...,2013-10-03 08:30:00+00:00,2013-10-03 08:45:00+00:00,285.0
3,2ace0a0c841190acb5513d4f011447a3e9df87da0cc213...,2013-10-03 08:00:00+00:00,2013-10-03 09:00:00+00:00,330.0
4,2ee325e10e1eec919fc5bfc4d64ed9951cbab4fb13b3e2...,2013-10-03 12:30:00+00:00,2013-10-03 12:30:00+00:00,165.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
# q_3.hint()
# q_3.solution()

---




**Allen**